## Frequency Response Functions

In [ ]:
# Setup properties including viscosity
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 200

In [ ]:
# System settings, all in SI

# Fluid + constants
g  = 9.81 # Gravity
nu = 1e-6 # Kinematic viscosity

#Setup
H = 0.02 # Plate separation
L = 0.80 # Channel length

wl = 0.05 # Width at piston
ll = 0.50 # Length at piston
wpreal = 0.04 # Piston width
lp = 0.30 # Piston length
wp = wpreal * lp / ll# Effective piston width to regain 1D problem
wr = 0.05 # Width at free surface

omega_n = np.sqrt(g*H/L * (1/wr+1/(wl-wp))) # Natural frequency

# Functions
def stokesboundary(omega):
    return np.sqrt(2*nu/omega)

def uexact_amp(z,delta,omega):
    uexact_amp = 1-np.cos((-1+1j)*z/delta)/np.cos((-1+1j)*H/(2*delta))
    uexact_amp = uexact_amp * g/L * wp/(wl-wp) * A * omega
    uexact_amp = uexact_amp * (-1j)/(omega_n**2-omega**2+(1+1j)*omega_n**2*delta/H*np.tan((-1+1j)*H/(2*delta)))
    return uexact_amp


print('Eigenfrequency of the system = ',format(omega_n,'.2f'),' rad/s')
print('Eigenfrequency of the system = ',format(omega_n/(2*np.pi),'.2f'),' Hz')
print('done')

In [ ]:
#Approximation, frequency response functions

#User choices
A = 0.02 #Piston amplitude
f = np.logspace(-1.5,1.5,int(1e5))    #Hz
omega = 2*np.pi*f #Driving frequency

d = stokesboundary(omega)

uexact = uexact_amp(0,d,omega)
Umax = np.abs(uexact)

phi = np.arctan(np.imag(uexact)/np.real(uexact))

amax = Umax*omega #Maximum acceleration between plates
Gamma = amax/g

Lmax = Umax/omega #Oscillation amplitude

print('omega_n =',format(omega_n,'.3f'),'rad/s = ',format(omega_n/(2*np.pi),'.3f'),'Hz')

plt.figure()
plt.subplot(2,1,1)
plt.loglog(f,Umax)
plt.xlabel('f [Hz]')
plt.ylabel('Umax [m/s]')
plt.grid()
plt.subplot(2,1,2)
plt.semilogx(f,phi/np.pi*180)
plt.semilogx(f,f/f*90,'k:')
plt.semilogx(f,f/f*-90,'k:')
plt.xlabel('f [Hz]')
plt.ylabel('phase [deg]')
plt.grid()
plt.show()

plt.figure()
plt.subplot(2,1,1)
plt.loglog(f,Gamma)
plt.xlabel('f [Hz]')
plt.ylabel('amax/g [-]')
plt.grid()
plt.subplot(2,1,2)
plt.loglog(f,Lmax)
plt.xlabel('f [Hz]')
plt.ylabel('Lmax [m]')
plt.grid()
plt.show()

In [ ]:
#Exact spatial profiles

def fplus(H,d):
    fplus = (np.sin(H/d)+np.sinh(H/d)) / (np.cos(H/d)+np.cosh(H/d))
    return fplus

def fmin(H,d):
    fmin = (-np.sin(H/d)+np.sinh(H/d)) / (np.cos(H/d)+np.cosh(H/d))
    return fmin    

def spatial(H,d,y):
    spatial = 1
    spatial = spatial -       (2*np.cos(H/(2*d))*np.cos(y/d)*np.cosh(H/(2*d))*np.cosh(y/d)+2*np.sin(H/(2*d))*np.sin(y/d)*np.sinh(H/(2*d))*np.sinh(y/d)) / (np.cos(H/d)+np.cosh(H/d))
    spatial = spatial + 1j * ((2*np.cos(y/d)*np.cosh(y/d)*np.sin(H/(2*d))*np.sinh(H/(2*d))-2*np.cos(H/(2*d))*np.cosh(H/(2*d))*np.sin(y/d)*np.sinh(y/d)) / (np.cos(H/d)+np.cosh(H/d)))
    return spatial

def uexact(t,y,omega):
    uexact = g/L * wp/(wl-wp) * (A*omega)
    uexact = uexact / (omega_n**2*(1-d/H*fplus(H,d)) - omega**2 + 1j*omega_n**2*d/H*fmin(H,d))
    uexact = uexact * spatial(H,d,y)
    uexact = uexact * np.exp(1j*omega*t)
    uexact = np.real(uexact)
    return uexact

#User choices
A = 0.02 #Piston amplitude
f = 0.5    #Hz
omega = 2*np.pi*f #Driving frequency

d = stokesboundary(omega)

y = np.linspace(-H/2,H/2,int(1e3))

plt.figure(figsize=(12,6))
plt.title('Exact profiles')
for t in np.linspace(0,2*np.pi/omega,13):

    u = uexact(t,y,omega)

    plt.plot(y,u,label=format(t,'.2f')+'s')
    
plt.xlabel('y [m]')
plt.ylabel('u [m/s]')
plt.legend()
plt.grid()
plt.show()